In [2]:
#allow output from every line
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
from pybaseball import lahman
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib notebook
import seaborn as sns
import plotly.express as px #used to create figures
from jupyter_dash import JupyterDash #used to create the dashboard, next four lines are for full dash functionality
import dash_core_components as dcc 
import dash_html_components as html
import plotly.graph_objects as go
from dash.dependencies import Input, Output

In [3]:
batting_data = pd.read_csv('Batting_Data.csv')
batting_data["Name"] = batting_data["Name"].str.replace("*","")
batting_data["Name"] = batting_data["Name"].str.replace("#","")
batting_data["Singles"] = batting_data["H"]- batting_data["2B"]-batting_data["3B"]-batting_data["HR"] #creates a singles column
batting_data['Singles'] = pd.to_numeric(batting_data['Singles'],errors='coerce') 
#makes singles a numeric variable
batting_data['Singles_Percentile'] = batting_data['Singles'].rank(pct = True)
#creates a percentile version of the singles column

In [13]:
# Build App

categories = ["H", "2B", "3B", "HR", "AB", "RBI", "SF"] #creates categories for radar plot
fig1=go.Figure() #creates blank plot
fig1.add_trace(go.Scatterpolar( #creates scatter plot structure 
    r=[0.5, 0.5,0.5, 0.5, 0.5, 0.5, 0.5], #adds league average percentiles as values for each category
    theta=categories, #sets categories to radar plot
    fill='toself', #visual effect
    name="League Average")) #names the average plot
fig1.update_layout( #visual effects
    polar=dict(
        radialaxis=dict( 
            visible=False, #hides radial axis
            range=[0,1])), #sets the range of the radar plot from 0 to 1, as all percentiles fall between these values
    showlegend=True) #shows legend for the radar plot

#fig3=go.Figure() #creates blank plot eventually used for pie chart

#app structural stuff
app = JupyterDash(__name__) 
app.layout = html.Div([
    html.H1("SAL384 Project: Player Comparison App"), #inserts title
    html.Label([
        "Player:",#places text before input box 
        dcc.Input( #creates input box
            id='name_input', type='text', #sets id for input box and limits input to text
            placeholder='Enter Player Name' #placeholder value for input box
            )
    ]),
    html.P('Radar Plot: Comparing 7 player batting statistic percentiles vs. league average (excl. Pitchers).'), #adds description of radar plot
    html.P('Pie Chart: Displaying player hit type distribution.'), #adds description of pie chart
    html.P('Scatter Plot: Comparing player slugging percentage against runs scored. Player selected highlighted in red.'), #adds description of scatter plot
        dcc.Graph(figure=fig1, id="fig1", style={'display':'inline-block', 'width':'40%'}),#radar plot 
        #inline block allows for figures to be side by side 
        #width changes their size such that they actually fit on the same line
        dcc.Graph(figure=fig3, id="fig3", style={'display': 'inline-block', 'width':'40%'}),#pie chart
        dcc.Graph(figure=fig2, id="fig2", #scatter plot format
             config={
                 #scatter plot options
                 'staticPlot':False, #allows for zoom
                 'scrollZoom':True, #can zoom using scroll wheel
                 'doubleClick':'reset', #double clicking on the scatter plot resets it
                 'showTips':True, #shows a tip that tells users about the double click
                 'displayModeBar':True,
                 'watermark':True
             })
])


@app.callback(                #callback for radar plot, calls on id
    Output('fig1', 'figure'), #output is the radar plot
    [Input("name_input", "value")] #input is a user typing a player's name
)

def update_figure(Player):   #updates the radar plot, takes 
    fig1=go.Figure() #creates blank plot
    fig1.add_trace(go.Scatterpolar( #create radar plot
        r=[0.5, 0.5,0.5, 0.5, 0.5, 0.5, 0.5], #average values 
        theta=categories, #adds categories to radar plot
        fill='toself', #visual effect
        name="League Average")) #names average plot
    fig1.update_layout( #updates radar plot
        polar=dict(
        radialaxis=dict(
        visible=False, #hides radial axis
        range=[0,1])), #sets the range of the radar plot from 0 to 1, as all percentiles fall between these values
        showlegend=True) #shows legend for the radar plot
    if(len(batting_data[batting_data['Name'] == Player]) > 0): #if statement that prevents plots from showing up unless a player's full name is entered
        x_row=batting_data[batting_data['Name'] == Player].index[0] #gets row number associated with player name
        player_hits=batting_data['H_percentile'][x_row] #gets player hits percentile
        player_doubles=batting_data['2B_percentile'][x_row]#gets player doubles percentile
        player_triples=batting_data['3B_percentile'][x_row]#gets player triples percentile
        player_hrs=batting_data['HR_percentile'][x_row]#gets player home runs percentile
        player_AB=batting_data['AB_percentile'][x_row]#gets player at bats percentile
        player_RBI=batting_data['RBI_percentile'][x_row]#gets player RBIs percentile
        player_SF=batting_data['SF_percentile'][x_row]#gets player sac flies percentile
        fig1.add_trace(go.Scatterpolar( #adds player values to radar plot
            r=[player_hits, player_doubles, player_triples, player_hrs, player_AB,
              player_RBI, player_SF], 
            theta=categories,
            fill='toself',
            name=Player))
    return(fig1) #returns completed radar plot


@app.callback( #Define callback to update scatter plot
    Output('fig2', 'figure'), #output is the scatter plot
    [Input("name_input", "value")]
)

def update_scatter(Player): #updates scatter plot
    fig2=go.Figure() #creates a blank figure
    if(len(batting_data[batting_data['Name'] == Player]) > 0): #if statement that prevents plots from showing up unless a player's full name is entered
        x_row=batting_data[batting_data['Name'] == Player].index[0] #gets row number associated with player name
        batting_data["Ind"] = (batting_data["Name"] == Player).astype(str) #player indicator column to highlight player mark
        fig2=px.scatter(batting_data, x='SLG', y='R', hover_data=["Name", "Team", "Pos"], color="Ind") #creates scatter plot
        fig2.update_layout(showlegend=False) #hides legend
    return(fig2) #returns scatter plot
#Run app and display result inline in the notebook
app.run_server(debug=True, use_reloader=False)

@app.callback( #Define callback to update pie chart
    Output('fig3', 'figure'), #output is the pie chart
    [Input("name_input", "value")]
)

def update_pie(Player): #updates pie chart
    if(len(batting_data[batting_data['Name'] == Player]) > 0):  #if statement that prevents plots from showing up unless a player's full name is entered
        y_row=batting_data[batting_data['Name'] == Player].index[0]  #gets row number associated with player name
        labels=['Singles', 'Doubles', 'Triples', 'Home Runs'] #sets labels for different sections of pie chart
        values= [batting_data['Singles'][y_row], batting_data['2B'][y_row], batting_data['3B'][y_row], batting_data['HR'][y_row]] #gets player values onto pie chart
        fig3=go.Figure(data=[go.Pie(labels=labels, values=values)]) #creates pie chart
        return(fig3) #returns pie chart
app.run_server() #runs app

OSError: Address 'http://127.0.0.1:8050' already in use.
    Try passing a different port to run_server.